<a href="https://colab.research.google.com/github/omlad93/cepstrum_seminar/blob/main/Cepstrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Seminar on DSP Algorithms and Applications**
## Introduction and Demonstaration of Cepstrum Analysis
In this notebook we will Explore the Fundemtals ideads and consepts 

### Initialzing notebook

In [ ]:
!pip install acoustics > null

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.collections as collections
%matplotlib inline
plt.style.use('bmh')
import numpy as np
from math import pi,e,ceil
import soundfile as sf
from google.colab import drive
from scipy import signal
from scipy.io.wavfile import read as load_sound
import acoustics.cepstrum as cep

drive.mount('/content/gdrive', force_remount=True)
dsp_dir = "/content/gdrive/MyDrive/4th year/DSP Seminar"

Mounted at /content/gdrive


In [ ]:
# Parameters
n = 20
time_xlim = (0,0.2)
plt.rcParams['figure.figsize'] = [50, 18]

In [ ]:
def sine_wave(f0:float,t:np.ndarray, phase:float=0.0):
    return np.sin(2 * np.pi * f0 * t + phase)

def sum_of_sines(f0:float,t:np.ndarray, n:int):
    waveform = np.zeros((t.shape[0],), dtype=float)
    elements = [
                (f0 / f) * sine_wave(f0=f,t=t,phase=f)
                for f in [f0*i for i in range(1,n+1)]
                ]
    info = [
            (2*np.pi*f,f)
            for  f in [f0*i for i in range(1,n+1)]
    ]
    for ele in elements:
      waveform += ele
    return waveform , elements ,info

def convert_to_mel(frequencies):
  return 2410*(2410*np.log(1+frequencies/625))

def pseudo_power_cepstrum(f:np.ndarray, in_mel:bool=False)->tuple:
  '''
  Returns quefrencies axis and power of f(t)
  '''
  log_fft_sq = np.log(np.square(np.absolute(np.fft.rfft(np.hamming(len(f)) * f))))
  pc = (np.abs(np.fft.rfft(log_fft_sq)))
  frequencies =np.fft.rfftfreq(len(f)) if not in_mel else convert_to_mel(np.fft.rfftfreq(len(f)))
  df = frequencies[1] - frequencies[0]
  quefrencies = np.fft.rfftfreq(log_fft_sq.size, df)
  return quefrencies,pc

def cepstrum_f0_detection(quefrency_vector:np.ndarray, cepstrum:np.ndarray, fmin=82, fmax=640):
    """Returns f0 based on cepstral processing."""
    valid = (quefrency_vector > 1/fmax) & (quefrency_vector <= 1/fmin)
    max_quefrency_index = np.argmax(np.abs(cepstrum)[valid])
    f0 = 1/quefrency_vector[valid][max_quefrency_index]
    return f0

def fourier(f,freq):
  fft = (np.abs(np.fft.rfft(np.hamming(len(f)) * f)))
  frequencies = np.fft.rfftfreq(len(f), 1/freq)
  return frequencies,fft

def liftering(signal,low=0,high=-1):
  for i in range(len(signal)):
    if i not in range(low,high+1):
      signal[i]=0
  return signal


## Creation of Waves in Time
### Creating Independt sine waves
generating sin(ωt+ω)

In [ ]:
FREQ = 1_000 # Hz
SAMPLES = ceil(1_000*4*pi) 
F0 = 0.1592
time= np.arange(SAMPLES) / FREQ


sum, all, info = sum_of_sines(F0 ,time, n)
fig, axes = plt.subplots(nrows= ceil(n//2), ncols=2)
fig.tight_layout(pad=3.0)
for i in range(len(all)):
  axes[i//2][i%2].plot(time,all[i] )
  axes[i//2][i%2].set_title(f'sin({info[i][0]:.0f}t+{info[i][1]:.2f}) in Time')

### Creation of wave with harmonics (sum of sine waves)
creating $f(t)=\sum_{i=1}^{n}sin(i𝜔_0t_𝜔)$  

In [ ]:
DURATION = 0.628
FREQ = 22_050
SAMPLES = int(FREQ*DURATION)
t = np.arange(SAMPLES) / FREQ
time= np.arange(SAMPLES) / FREQ
F0 =500


sum, all, info = sum_of_sines(F0 ,time, n)
fig, ax = plt.subplots()
ax.plot(time, sum)
ax.set_xlabel('time (s)')
ax.set_title('Harmonic (sum of sines) in Time')

### Fourier Transform
### Creating FT of Independt sine waves
generating $F(sin(i𝜔_0+𝜔))$ for i in range(n)   

In [ ]:
fourier_abs_sines = []
sum, all, info = sum_of_sines(F0 ,time, n)
fig, axes = plt.subplots(nrows=n//2, ncols=2,)

for i in range(len(all)):
  abs_fourier = np.abs(np.fft.fft(all[i]))
  f_axis = np.fft.fftfreq(SAMPLES)*2*pi
  fourier_abs_sines.append(abs_fourier)
  ax = axes[(i%(n//2))][(i//(n//2))]
  ax.plot(f_axis ,abs_fourier)
  ax.set_title(f'F(sin({info[i][0]:.1f}t+{info[i][1]:.0f})) in Frequency [Rad/s]')
  ax.tick_params(axis='x', which='major', labelsize=15)
plt.setp(axes , 
         xticks=      [-3.14 ,-1.57 ,0    , 1.57 , 3.14], 
         xticklabels= ['-π'  ,'-π/2' ,0 , 'π/2' , 'π'  ])
fig.tight_layout(pad=3.0)


### creating log(FT of sum of sines)
creating $F(\sum_{i=1}^{n}sin(i𝜔_0t_𝜔))$ and $log(F(\sum_{i=1}^{n}sin(i𝜔_0t_𝜔)))$

In [ ]:
frequencies = np.fft.fftshift(np.fft.fftfreq(SAMPLES)*2*pi)
fft = np.abs(np.fft.fft(np.hamming(SAMPLES) * sum))
log_fft_sq = np.log(np.square(fft))
fourier_of_sum = np.log(np.abs(fft))

fig, ax = plt.subplots(nrows=2)
plt.setp(axes , 
         xticks=      [-3.14 ,-1.57 ,0    , 1.57 , 3.14], 
         xticklabels= ['-π'  ,'-π/2' ,0 , 'π/2' , 'π'  ])
fig.suptitle('Fourier', fontsize=40)
ax[0].plot(frequencies, fft)
ax[0].set_xlabel('frequency (Rad/s)', fontsize=20)
ax[0].set_title('F(f(t))', fontsize=20)
ax[1].plot(frequencies, log_fft_sq)
ax[1].set_xlabel('frequency (Rad/s)', fontsize=20)
ax[1].set_title('log(F{f(t)}^2)', fontsize=20)



### creating Cepstrum
creating $F^{-1}(log(F(\sum_{i=1}^{n}sin(i𝜔_0t_𝜔)^2)))^2$  
Extracting fundemental frequency from cepstrum


In [ ]:
epsilon = 10e-4
sum, all, info = sum_of_sines(F0 ,time, n)

quefrencies = np.fft.fftfreq(fft.size)
sum_cepstrum,fs = cep.complex_cepstrum(sum)
# print(sum_cepstrum)
fig, ax = plt.subplots()
ax.plot(quefrencies, sum_cepstrum)
ax.set_xlabel('quefrency (s)', fontsize=20)
ax.set_title('C(f(t))', fontsize=20)
ax.set_xlim(-epsilon, max(quefrencies)+0.1)

## Testing Applicability
We will now see two features of of cepstral analysis  
1. simplication of convolution in time: $C(f(t)*h(t)) ≈ C(f(t))+C(h(t))$
2. advantges of **liftering**: the cepstral equivalent of spectrum filtering: echo canceling


In [ ]:
k,l = 100,500
SIG = np.asarray([k/(n+k)*np.sin(n/5) for n in range(l)])
CON = np.convolve(SIG,SIG)
SIG_PADDED = np.pad(SIG, [1, CON.size - SIG.size-1],'constant') # Zero Padded in the End
t= np.arange(SIG_PADDED.size)

sig,_ = cep.complex_cepstrum(SIG)
fig, ax = plt.subplots(nrows=2,sharex=True)
ax[0].plot(t, SIG_PADDED)
ax[0].set_xlabel('Time (s)', fontsize=20)
ax[0].set_title('Pulses', fontsize=20)
ax[1].plot(t, CON)
ax[1].set_xlabel('Time (s)', fontsize=20)
ax[1].set_title('Pulses', fontsize=20)

In [ ]:
sig, _ = cep.complex_cepstrum(SIG_PADDED)
con, _ = cep.complex_cepstrum(CON)
f = np.fft.fftfreq(SIG_PADDED.size,1/(2*pi))
q = np.fft.fftshift(np.fft.fftfreq(f.size))
sum = sig+sig
fig, ax = plt.subplots(nrows=3)
plt.setp(ax , 
         xlabel = 'Cepstrum by quefrency')
ax[0].plot(q, sum)
ax[1].plot(q, con)
ax[2].plot(q, sum)
ax[2].plot(q, con, 'r+')